In [ ]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_fpath = data_dir / "2022-07-bmvc-report-01-data00-scheduler-params.csv"

import pandas as pd
df = pd.read_csv(data_fpath) 

def check_df(df):
    print(df.shape)
    print(df.dtypes)
    return df.head(5)

check_df(df)

In [ ]:
data_df = pd.read_csv(data_fpath) 
df = data_df
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: float(x.lstrip("[").rstrip("]")))
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: {0.985000: "higher decay", 0.998500: "equiv. decay"}[x])
df.drop(columns=["Name"], inplace=True)
check_df(df)

In [ ]:
data_df = df

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

def get_color_mask_compare_by_line_every_n_cols(pt, n):
    
    if n > 2:
        raise NotImplemented("n > 2 is not implemented ===> deal with color scale!")
    
    assert pt.shape[1] % n == 0, f"pt.shape[1] % n != 0: {pt.shape[1] % n}"

    def get_ordering_signal(arr: np.ndarray) -> np.ndarray:
        """at each line, the values are assigned a value from 0 to 1 respecting the ordering, such that 0 is the min and 1 is the max"""
        # it's ok to round up to 3 decimals because i will probably never have more than 1001 values
        # and it makes the table easier to read/debug
        assert arr.shape[1] < 1001, f"arr.shape[1] < 1001: {arr.shape[1]}"
        return np.round(np.argsort(arr) / (n - 1), decimals=3)

    return np.concatenate(
        [   
            get_ordering_signal(pt.iloc[:, i * n:(i + 1) *n].values)
            for i in range(pt.shape[1] // n)
        ],
        axis=1,
    )
    
COLOR_0 = 'background-color: #DC143C55'
COLOR_1 = 'background-color: #228B2255'
    
def color_from_mask(color_mask: np.ndarray):
    
    def color(_):
        colormap = np.empty_like(color_mask, dtype=object)
        colormap[color_mask == 0] = COLOR_0
        colormap[color_mask == 1] = COLOR_1
        return colormap
    
    return color

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

df = data_df.copy()

row_cols = ["mvtec_class_type", "normal_class_fullqualified"]
column_cols = ["supervise_mode", "loss", "scheduler_parameters"]

metrics = {
    # "test/avg-precision": ["mean", "std"],
    # "validate/avg-precision-histavg": ["mean",],
    "test/roc-auc": ["mean", "std"],
    "validate/roc-auc-histavg": ["mean",],
}
metric_cols = [(k, v) for k, list_vals in metrics.items() for v in list_vals]

pt_all_metrics = pd.pivot_table(df, index=row_cols, columns=column_cols, aggfunc=metrics)

# when it's a percentage
pt_all_metrics = 100 * pt_all_metrics

# pt_test_avg = pt_all_metrics[("test/avg-precision", "mean")]
pt_test_avg = pt_all_metrics[("test/roc-auc", "mean")]
pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
pt_test_avg.loc[("", "texture mean"), :] = pt_test_avg.loc["texture"].mean(axis=0)
pt_test_avg.loc[("", "all mean"), :] = pt_test_avg.mean(axis=0)

# pt_test_std = pt_all_metrics[("test/avg-precision", "std")]
pt_test_std = pt_all_metrics[("test/roc-auc", "std")]
pt_test_std.loc[("", "object mean"), :] = pt_test_std.loc["object"].mean(axis=0)
pt_test_std.loc[("", "texture mean"), :] = pt_test_std.loc["texture"].mean(axis=0)
pt_test_std.loc[("", "all mean"), :] = pt_test_std.mean(axis=0)

# pt_validate_histmean_avg = pt_all_metrics[("validate/avg-precision-histavg", "mean")]
pt_validate_histmean_avg = pt_all_metrics[("validate/roc-auc-histavg", "mean")]
pt_validate_histmean_avg.loc[("", "object mean"), :] = pt_validate_histmean_avg.loc["object"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "texture mean"), :] = pt_validate_histmean_avg.loc["texture"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "all mean"), :] = pt_validate_histmean_avg.mean(axis=0)

pt_composed = pt_test_avg.applymap("{:.1f}".format) + " " + pt_test_std.applymap("({:.1f})".format) + " " + pt_validate_histmean_avg.applymap("[{:.1f}]".format)

# compare all

In [ ]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

# fixed loss

## old-fcdd

In [ ]:
pt = pt_composed.loc[:, (slice(None), "old-fcdd")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), "old-fcdd")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

## pixelwise-batch-avg

In [ ]:
pt = pt_composed.loc[:, (slice(None), "pixelwise-batch-avg")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), "pixelwise-batch-avg")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

# fixed decay

## higher decay

In [ ]:
pt = pt_composed.loc[:, (slice(None), slice(None), "higher decay")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), slice(None), "higher decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

## equiv. decay

In [ ]:
pt = pt_composed.loc[:, (slice(None), slice(None), "equiv. decay")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), slice(None), "equiv. decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

In [ ]:
pt = pt_composed.loc[:, (slice(None), "old-fcdd", "equiv. decay")].copy()
pt
pt_color_from = pt_test_avg.loc[:, (slice(None), "old-fcdd", "equiv. decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
# pt.style.apply(color_from_mask(color_mask), axis=None)